In [46]:
import pandas as pd
import pyreadstat
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
import utils

%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [48]:
df_sch, meta_sch = pyreadstat.read_sav('data/CY08MSP_STU_QQQ.sav')

In [84]:
import os
with open('data/openai.api.key', 'r') as filek: 
    openai_key = filek.read()
os.environ["OPENAI_API_KEY"] =  openai_key 

In [56]:
   
cols = utils.meta2docs(meta_sch, excluded=['test', 'CNTRYID'])

cols_vectorstore = Chroma.from_documents(cols, OpenAIEmbeddings())

cols_retriever = cols_vectorstore.as_retriever()

In [35]:
test_data = [
    {'question':'Which country has the highest average  teacher student ratio? How much is it? ','expected_columns': ['STRATIO', 'CNT'], 'expected_answer': 28.95},
    {'question':'Which country has the highest average class sizes? How much is it?','expected_columns': ['CLSIZE', 'CNT'], 'expected_answer': 42.65 },    
    {'question':'In how many languages ​​was the questionnaire filled out?', 'expected_columns': ['LANGTEST_QQQ'], 'expected_answer': 54},
    {'question':'What is Hungary average teacher-student ratio?', 'expected_columns': ['STRATIO', 'CNT'], 'expected_answer': 10.83},
    {'question':'What is OECD average teacher-student ratio?', 'expected_columns': ['STRATIO', 'OECD'], 'expected_answer':  12.551608},
    {'question':'Where is average teacher-student ratio in Hungary compared to the OECD average in terms of average teacher-student ratio? Give the difference as number.','expected_columns': ['STRATIO', 'OECD', 'CNT'], 'expected_answer': -1.721608 },    
    #{'question':'how many education systems were involved in the survey? ','expected_columns': ['STRATIO', 'CNT'], 'expected_answer': 28.95},
    {'question':'how many countries were involved in the survey?','expected_columns': ['CNT'], 'expected_answer': 80},
    {'question':'how many countries that are OECD memeber were involved in the survey? ','expected_columns': ['OECD', 'CNT'], 'expected_answer': 37},
    {'question':'how many schools were involved in the survey?','expected_columns': ['CNTSCHID'], 'expected_answer': 21629},
    {'question':'what is the relation between school autonomy  and teacher participation? Please give linear regression coefficient as result','expected_columns': ['SCHAUTO','TCHPART'], 'expected_answer':  0.2216 },
]


In [58]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0)

In [68]:
cols_retriever.invoke('1. Student ID')

[Document(page_content='Intl. Student ID', metadata={'original_col_name': 'CNTSTUID', 'year': 2022}),
 Document(page_content='Intl. Student ID', metadata={'original_col_name': 'CNTSTUID', 'year': 2022}),
 Document(page_content='Intl. Student ID', metadata={'original_col_name': 'CNTSTUID', 'year': 2022}),
 Document(page_content='Intl. School ID', metadata={'original_col_name': 'CNTSCHID', 'year': 2022})]

In [75]:
utils.pipeline(llm, 'How many students were interviewed?', df_sch, meta_sch, cols_retriever)

1. Student ID
2. Interview Date
3. Interview Time
4. Interviewer Name
5. Interview Location
6. Interview Duration
7. Interview Outcome (e.g. accepted, rejected)
8. Interview Feedback
['Intl. Student ID', 'Intl. Student ID', 'Intl. Student ID', 'Intl. School ID']
['Intl. Student ID', 'Intl. School ID']
['Teacher Questionnaire Option', 'Teacher Questionnaire Option', 'Teacher Questionnaire Option', 'Language of Questionnaire']
['Teacher Questionnaire Option', 'Language of Questionnaire']
['How much time spent: Look for practical information online (e.g. find a place, book a train ticket, buy a product)', 'How much time spent: Look for practical information online (e.g. find a place, book a train ticket, buy a product)', 'How much time spent: Look for practical information online (e.g. find a place, book a train ticket, buy a product)', "During admission, how often consider: Student's working status"]
['How much time spent: Look for practical information online (e.g. find a place, book a 

{'question': 'How many students were interviewed?',
 'result': 0.0,
 'hint_cols': ['CNTSTUID',
  'CNTSCHID',
  'Option_TQ',
  'LANGTEST_QQQ',
  'IC177Q04JA',
  'SC012Q11JA',
  'Option_TQ',
  'LANGTEST_QQQ',
  'SC012Q06TA',
  'REGION',
  'DURECEC',
  'SC037Q09TA',
  'ST330Q03WA',
  'Option_TQ',
  'SC037Q07TA',
  'TEAFDBK']}

df_sch['CNTSTUID'].nunique()

In [77]:
float(df_sch['CNTSTUID'].nunique())

613744.0

In [78]:
df_sch['CNTSTUID'].dropna()

0           800001.0
1           800002.0
2           800003.0
3           800005.0
4           800006.0
             ...    
613739    86007488.0
613740    86007489.0
613741    86007490.0
613742    86007491.0
613743    86007492.0
Name: CNTSTUID, Length: 613744, dtype: float64

In [79]:
df_sch.dropna()

,CNT,CNTRYID,CNTSCHID,CNTSTUID,CYC,NatCen,STRATUM,SUBNATIO,REGION,OECD,...,PV4MPRE,PV5MPRE,PV6MPRE,PV7MPRE,PV8MPRE,PV9MPRE,PV10MPRE,SENWT,VER_DAT,test
